In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
DATASET_CSABSTRUCT_DIR = './gdrive/Shareddrives/DATASETS/CSAbstruct/'
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

In [ ]:
!pip install transformers -q
!pip install sentence_transformers -q
!pip install datasets -q

     |████████████████████████████████| 4.7 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.0 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 101 kB 9.0 MB/s 
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 1.3 MB 18.7 MB/s 
     |████████████████████████████████| 365 kB 9.4 MB/s 
     |████████████████████████████████| 115 kB 65.0 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 141 kB 67.1 MB/s 
     |████████████████████████████████| 127 kB 73.2 MB/s 


In [ ]:
import pandas as pd

In [ ]:
df_test = pd.read_parquet(DATASET_CSABSTRUCT_DIR + 'test.parquet')

In [ ]:
df_test.columns

Index(['id', 'sentence', 'subject_label', 'label_id'], dtype='object')

In [ ]:
# same text but with different labels
print(f'{len(df_test.index)}')
mask = df_test.groupby('sentence')['label_id'].transform('nunique') > 1
df_test = df_test[~mask].copy()
print(f'{len(df_test.index)}')

1349
1349


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', model_max_length=512)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

model_path = './gdrive/Shareddrives/MODELS/csabstruct_classification/'
config = AutoConfig.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.to(device)
model.device

device(type='cuda', index=0)

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0)

In [ ]:
predictions = pipe(list(df_test['sentence']))

In [ ]:
y_pred = []
for pred in predictions:
  y_pred.append(config.label2id[pred['label']])

In [ ]:
y_test_true = list(df_test['label_id'])

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score

output_dict = {}
output_dict['f1_macro'] = f1_score(y_test_true, y_pred, average='macro')
output_dict['f1_micro'] = f1_score(y_test_true, y_pred, average='micro')
output_dict

{'f1_macro': 0.6729097813751009, 'f1_micro': 0.6641957005189029}

In [ ]:
config.id2label

{0: 'background', 1: 'objective', 2: 'method', 3: 'result', 4: 'other'}

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_true, y_pred, target_names=config.id2label.values()))

              precision    recall  f1-score   support

  background       0.83      0.65      0.73       493
   objective       0.57      0.46      0.51       155
      method       0.56      0.73      0.63       421
      result       0.64      0.67      0.65       219
       other       0.89      0.79      0.83        61

    accuracy                           0.66      1349
   macro avg       0.70      0.66      0.67      1349
weighted avg       0.69      0.66      0.67      1349

